### **Import Library**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from prophet import Prophet
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import logging # Untuk mengontrol pesan log Prophet
import plotly.express as px

# Matikan logger Prophet agar output tidak terlalu banyak
logging.getLogger('prophet').setLevel(logging.WARNING)

In [2]:
# pip install Prophet

### **Import Data Hasil Rules**

In [3]:
df = pd.read_csv('data/rules2024.csv')
df

,Unnamed: 0,antecedents,consequents,support,confidence,lift
0,0,"frozenset({'SAKINAH 1500ML', 'GREENFIELDS COKL...",frozenset({'GREENFIELDS STRAW 105ML'}),0.000109,1,4922.846154
1,1,"frozenset({'BISKUAT BOLU CKL 16GR', 'INDOMILK ...","frozenset({'INDOMILK KID VNL 115ML', 'CLOUD 9 ...",0.000141,1,2206.793103
2,2,"frozenset({'GATSBY SPL/C GS 175ML', 'FRESHCARE...",frozenset({'GATSBY SPL/C DO 175ML'}),0.000141,1,1185.129630
3,3,"frozenset({'KISPRAY POUCH GLAM/GLD 300ML', 'IN...",frozenset({'SOKLIN LIQ LAVENDER 625ML'}),0.000109,1,831.129870
4,4,"frozenset({'SEDAAP MIE CUP BASO BLEDUK', 'SEDA...",frozenset({'SEDAAP MIE CUP SOTO'}),0.000125,1,780.451220
5,5,"frozenset({'SEDAAP MIE CUP AYM JERIT /12', 'SE...",frozenset({'SEDAAP MIE CUP SOTO'}),0.000109,1,780.451220
6,6,"frozenset({'ATTACK JAZ 1 CINTA 1 6 KG', 'NUVO ...",frozenset({'NUVO SOAP BLUE 72G'}),0.000109,1,484.825758
7,7,"frozenset({'BISKUAT BOLU CKL 16GR', 'INDOMILK ...",frozenset({'CLOUD 9 VANILA 15 4G'}),0.000141,1,378.680473
8,8,"frozenset({'BISKUAT BOLU CKL 16GR', 'INDOMILK ...",frozenset({'CLOUD 9 VANILA 15 4G'}),0.000141,1,378.680473
9,9,"frozenset({'DELFI TOP TRIP/CHO 9GR', 'MOMOGI J...",frozenset({'DELFI TOP B/WHITE 9GR'}),0.000109,1,353.574586


In [4]:
# Pastikan semua elemen dalam kolom antecedents dan consequents adalah frozenset
df["antecedents"] = df["antecedents"].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)
df["consequents"] = df["consequents"].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)

# Mengambil semua item unik
all_items = set()
for col in ["antecedents", "consequents"]:
    for items in df[col]:
        all_items.update(items)

all_items

{'ADEM SARI CHINGKU MD LMN 350ML',
 'ATTACK JAZ 1 CINTA 1 6 KG',
 'BISKUAT BOLU CKL 16GR',
 'CLOUD 9 CHOCO 15 4G',
 'CLOUD 9 VANILA 15 4G',
 'DAIA LEMON 65GR',
 'DELFI TOP B/WHITE 9GR',
 'DELFI TOP CHO 9GR',
 'DELFI TOP STR 9GR',
 'DELFI TOP TRIP/CHO 9GR',
 'EKONOMI LIQUID REFF 235ML',
 'FRESHCARE ORIGINAL 10ML',
 'GATSBY SPL/C DO 175ML',
 'GATSBY SPL/C GS 175ML',
 'GREENFIELDS COKLAT 105ML',
 'GREENFIELDS STRAW 105ML',
 'INDOMIE GRG SPECIAL',
 'INDOMIE RAMEN TORI KARA/20',
 'INDOMIE SOTO MIE',
 'INDOMILK KID CKL 115ML',
 'INDOMILK KID STR 115ML',
 'INDOMILK KID VNL 115ML',
 'KISPRAY POUCH GLAM/GLD 300ML',
 'MAMA LEMON 115ML',
 'MOMOGI JGG BKR 6GR',
 'MR POTATO CRIPS ORI 35G',
 'NEXTAR BROWNIES 42GR',
 'NEXTAR PINEAPLE 112G',
 'NUVO SOAP BLUE 72G',
 'NUVO SOAP RED 72G',
 'ROMA KELAPA 300GR',
 'SAKINAH 1500ML',
 'SEDAAP MIE AYAM BWG',
 'SEDAAP MIE CUP AYM JERIT /12',
 'SEDAAP MIE CUP BASO BLEDUK',
 'SEDAAP MIE CUP BASO SPC',
 'SEDAAP MIE CUP KARE',
 'SEDAAP MIE CUP SOTO',
 'SEDAAP MIE S

### **Import Data Transaksi**

In [5]:
groceries = pd.read_excel('data/final_data2024.xlsx')
groceries

,TANGGAL,NO TRANSAKSI,NAMA BARANG,QTY
0,2024-01-01,2401011010001,GULA ROSE BRAND 1KG,1
1,2024-01-01,2401011010002,RIZKI MG 850ML/900ML BTL,3
2,2024-01-01,2401011010003,ULTRA MIMI VNL 125ML,1
3,2024-01-01,2401011010003,BEAR BRAND 189ML,1
4,2024-01-01,2401011010003,TINI WINI BITI ASIN 20 GR,1
...,...,...,...,...
167618,2024-12-31,2412311020122,WALLS POPULAIRE CKL,1
167619,2024-12-31,2412311020123,THERMAL 80X50 TRUST PAPER,1
167620,2024-12-31,2412311020124,SAKINAH 600ML,1
167621,2024-12-31,2412311020124,WALLS PP RAINBOW P,1


### **Filter Data Transaksi**

In [6]:
df_filtered = groceries[groceries['NAMA BARANG'].isin(all_items)]
df_filtered 

,TANGGAL,NO TRANSAKSI,NAMA BARANG,QTY
20,2024-01-01,2401011010011,INDOMIE GRG SPECIAL,1
32,2024-01-01,2401011010015,EKONOMI LIQUID REFF 235ML,1
55,2024-01-01,2401011010017,ROMA KELAPA 300GR,1
59,2024-01-01,2401011010018,INDOMILK KID VNL 115ML,1
73,2024-01-01,2401011010023,ROMA KELAPA 300GR,1
...,...,...,...,...
167537,2024-12-31,2412311020090,INDOMILK KID CKL 115ML,1
167552,2024-12-31,2412311020096,INDOMIE RAMEN TORI KARA/20,1
167607,2024-12-31,2412311020118,INDOMILK KID STR 115ML,1
167608,2024-12-31,2412311020118,INDOMILK KID VNL 115ML,1


In [7]:
# Menampilkan jumlah unik nama barang
jumlah_nama_barang = df_filtered['NAMA BARANG'].nunique()
print("Jumlah nama barang unik:", jumlah_nama_barang)

# Menampilkan isi nama barang yang unik
nama_barang_unik = df_filtered['NAMA BARANG'].unique()
print("Nama barang unik:")
for nama in nama_barang_unik:
    print("-", nama)

Jumlah nama barang unik: 45
Nama barang unik:
- INDOMIE GRG SPECIAL
- EKONOMI LIQUID REFF 235ML
- ROMA KELAPA 300GR
- INDOMILK KID VNL 115ML
- SEDAAP MIE AYAM BWG
- SEDAAP MIE SOTO
- ATTACK JAZ 1 CINTA 1 6 KG
- WALLS POPULAIRE CKL
- MAMA LEMON 115ML
- WALLS CORN STRW VANILA
- DELFI TOP CHO 9GR
- DELFI TOP STR 9GR
- DELFI TOP B/WHITE 9GR
- SEDAAP MIE CUP SOTO
- SEDAAP MIE CUP KARE
- DELFI TOP TRIP/CHO 9GR
- DAIA LEMON 65GR
- NUVO SOAP RED 72G
- NUVO SOAP BLUE 72G
- SEDAAP MIE CUP AYM JERIT /12
- SEDAAP MIE CUP BASO BLEDUK
- WALLS CORN B/WHITE 82GR
- BISKUAT BOLU CKL 16GR
- INDOMIE SOTO MIE
- SOKLIN LIQ LAVENDER 625ML
- INDOMILK KID STR 115ML
- WALLS POPULAIRE STR
- SAKINAH 1500ML
- KISPRAY POUCH GLAM/GLD 300ML
- MR POTATO CRIPS ORI 35G
- INDOMILK KID CKL 115ML
- GATSBY SPL/C DO 175ML
- GATSBY SPL/C GS 175ML
- FRESHCARE ORIGINAL 10ML
- INDOMIE RAMEN TORI KARA/20
- NEXTAR BROWNIES 42GR
- CLOUD 9 VANILA 15 4G
- TEH GELAS ORI 170ML
- CLOUD 9 CHOCO 15 4G
- NEXTAR PINEAPLE 112G
- SEDAAP MIE C

### **Agregasi Data Mingguan**

In [8]:
# Agregasi jumlah QTY per tanggal dan nama barang
df_grouped = df_filtered.groupby(['TANGGAL', 'NAMA BARANG'], as_index=False)['QTY'].sum()

tanggal_range = pd.date_range(start='2024-01-01', end='2024-12-31')
all_items = df_filtered['NAMA BARANG'].unique()

# Buat kombinasi semua tanggal dan item
multi_index = pd.MultiIndex.from_product([tanggal_range, all_items], names=['TANGGAL', 'NAMA BARANG'])
all_combinations = pd.DataFrame(index=multi_index).reset_index()

# Gabungkan dengan hasil groupby
df_joined = pd.merge(all_combinations, df_grouped, on=['TANGGAL', 'NAMA BARANG'], how='left')
df_joined['QTY'] = df_joined['QTY'].fillna(0)

# Pastikan TANGGAL dalam datetime
df_joined['TANGGAL'] = pd.to_datetime(df_joined['TANGGAL'])

# Urutkan dulu datanya
df_joined = df_joined.sort_values('TANGGAL')

# Ambil tanggal paling awal
start_date = df_joined['TANGGAL'].min()

# Hitung minggu ke-n secara manual (tiap 7 hari)
df_joined['WEEK_NUMBER'] = ((df_joined['TANGGAL'] - start_date).dt.days // 7) + 1

# Agregasi berdasarkan minggu dan nama barang
df_weekly = df_joined.groupby(['WEEK_NUMBER', 'NAMA BARANG'], as_index=False)['QTY'].sum()

# (Opsional) Tambahkan kolom tanggal mulai minggu
df_weekly['TANGGAL'] = start_date + pd.to_timedelta((df_weekly['WEEK_NUMBER'] - 1) * 7, unit='D')

# Visualisasi
fig = px.line(df_weekly, x='TANGGAL', y='QTY', color='NAMA BARANG',
              title='Jumlah Penjualan per Minggu (Setiap 7 Hari)')
fig.update_layout(legend_title_text='Klik untuk tampilkan/sembunyikan item')
fig.show()

In [9]:
df_weekly

,WEEK_NUMBER,NAMA BARANG,QTY,TANGGAL
0,1,ADEM SARI CHINGKU MD LMN 350ML,0.0,2024-01-01
1,1,ATTACK JAZ 1 CINTA 1 6 KG,10.0,2024-01-01
2,1,BISKUAT BOLU CKL 16GR,5.0,2024-01-01
3,1,CLOUD 9 CHOCO 15 4G,3.0,2024-01-01
4,1,CLOUD 9 VANILA 15 4G,4.0,2024-01-01
...,...,...,...,...
2380,53,TEH GELAS ORI 170ML,0.0,2024-12-30
2381,53,WALLS CORN B/WHITE 82GR,0.0,2024-12-30
2382,53,WALLS CORN STRW VANILA,0.0,2024-12-30
2383,53,WALLS POPULAIRE CKL,9.0,2024-12-30


### **Pivot Data**

In [10]:
df_pivoted= df_weekly.pivot(index='TANGGAL', columns='NAMA BARANG', values='QTY')
df_pivoted.columns.name = None  # hilangkan nama kolom atas
df_pivoted = df_pivoted.reset_index() 
df_pivoted

,TANGGAL,ADEM SARI CHINGKU MD LMN 350ML,ATTACK JAZ 1 CINTA 1 6 KG,BISKUAT BOLU CKL 16GR,CLOUD 9 CHOCO 15 4G,CLOUD 9 VANILA 15 4G,DAIA LEMON 65GR,DELFI TOP B/WHITE 9GR,DELFI TOP CHO 9GR,DELFI TOP STR 9GR,...,SEDAAP MIE CUP BASO SPC,SEDAAP MIE CUP KARE,SEDAAP MIE CUP SOTO,SEDAAP MIE SOTO,SOKLIN LIQ LAVENDER 625ML,TEH GELAS ORI 170ML,WALLS CORN B/WHITE 82GR,WALLS CORN STRW VANILA,WALLS POPULAIRE CKL,WALLS POPULAIRE STR
0,2024-01-01,0.0,10.0,5.0,3.0,4.0,5.0,6.0,8.0,5.0,...,0.0,2.0,5.0,38.0,2.0,3.0,17.0,2.0,10.0,4.0
1,2024-01-08,10.0,0.0,4.0,7.0,1.0,2.0,7.0,4.0,4.0,...,2.0,3.0,1.0,16.0,2.0,5.0,12.0,4.0,3.0,4.0
2,2024-01-15,8.0,0.0,7.0,5.0,7.0,0.0,3.0,1.0,3.0,...,2.0,3.0,2.0,20.0,0.0,4.0,3.0,0.0,7.0,7.0
3,2024-01-22,2.0,1.0,5.0,5.0,7.0,2.0,4.0,2.0,0.0,...,2.0,2.0,0.0,32.0,1.0,10.0,22.0,5.0,7.0,6.0
4,2024-01-29,0.0,5.0,0.0,10.0,7.0,3.0,9.0,10.0,5.0,...,2.0,0.0,2.0,16.0,2.0,9.0,8.0,2.0,8.0,5.0
5,2024-02-05,2.0,0.0,1.0,10.0,6.0,5.0,3.0,10.0,2.0,...,2.0,0.0,3.0,24.0,1.0,4.0,15.0,1.0,4.0,6.0
6,2024-02-12,1.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,6.0,...,2.0,3.0,2.0,16.0,0.0,5.0,3.0,2.0,9.0,7.0
7,2024-02-19,4.0,0.0,2.0,1.0,5.0,3.0,1.0,6.0,12.0,...,0.0,3.0,1.0,22.0,2.0,8.0,9.0,3.0,5.0,7.0
8,2024-02-26,1.0,6.0,3.0,2.0,3.0,0.0,6.0,17.0,9.0,...,2.0,3.0,3.0,20.0,2.0,6.0,6.0,3.0,5.0,2.0
9,2024-03-04,2.0,3.0,3.0,7.0,7.0,17.0,4.0,2.0,9.0,...,0.0,3.0,4.0,11.0,4.0,4.0,5.0,1.0,5.0,13.0


In [11]:
# 1. Pisahkan kolom tanggal dan kolom produk
produk_df = df_pivoted.drop(columns=["TANGGAL"])

# 2. Hitung jumlah 0 per kolom
zero_counts = (produk_df == 0).sum()

# 3. Ambil item yang jumlah 0-nya <= 5
filtered_items = zero_counts[zero_counts <= 5].index

# 4. Filter dataframe hanya dengan item yang lolos kriteria 0
filtered_df = produk_df[filtered_items]

# 5. Hitung total penjualan per item
total_sales = filtered_df.sum().sort_values(ascending=False).index

# 6. Ambil 15 item dengan total penjualan tertinggi
# top_3_items = total_sales.head(15).index

# 7. Final dataframe dengan 15 item terbaik
df_final = df_pivoted[["TANGGAL"] + total_sales.tolist()]

In [12]:
df_final.head()

,TANGGAL,INDOMIE GRG SPECIAL,INDOMILK KID CKL 115ML,SEDAAP MIE SOTO,SAKINAH 1500ML,INDOMILK KID VNL 115ML,INDOMILK KID STR 115ML,INDOMIE SOTO MIE,SEDAAP MIE AYAM BWG,EKONOMI LIQUID REFF 235ML,WALLS CORN B/WHITE 82GR,WALLS POPULAIRE CKL,WALLS POPULAIRE STR,MAMA LEMON 115ML,DELFI TOP CHO 9GR,ROMA KELAPA 300GR,DELFI TOP B/WHITE 9GR,BISKUAT BOLU CKL 16GR,DELFI TOP STR 9GR
0,2024-01-01,129.0,3.0,38.0,6.0,8.0,3.0,18.0,14.0,9.0,17.0,10.0,4.0,3.0,8.0,12.0,6.0,5.0,5.0
1,2024-01-08,119.0,12.0,16.0,6.0,13.0,14.0,13.0,1.0,6.0,12.0,3.0,4.0,6.0,4.0,8.0,7.0,4.0,4.0
2,2024-01-15,88.0,13.0,20.0,5.0,10.0,10.0,16.0,9.0,4.0,3.0,7.0,7.0,7.0,1.0,6.0,3.0,7.0,3.0
3,2024-01-22,147.0,18.0,32.0,8.0,14.0,16.0,19.0,3.0,6.0,22.0,7.0,6.0,8.0,2.0,3.0,4.0,5.0,0.0
4,2024-01-29,114.0,20.0,16.0,4.0,19.0,22.0,15.0,5.0,11.0,8.0,8.0,5.0,7.0,10.0,6.0,9.0,0.0,5.0


In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   TANGGAL                    53 non-null     datetime64[ns]
 1   INDOMIE GRG SPECIAL        53 non-null     float64       
 2   INDOMILK KID CKL 115ML     53 non-null     float64       
 3   SEDAAP MIE SOTO            53 non-null     float64       
 4   SAKINAH 1500ML             53 non-null     float64       
 5   INDOMILK KID VNL 115ML     53 non-null     float64       
 6   INDOMILK KID STR 115ML     53 non-null     float64       
 7   INDOMIE SOTO MIE           53 non-null     float64       
 8   SEDAAP MIE AYAM BWG        53 non-null     float64       
 9   EKONOMI LIQUID REFF 235ML  53 non-null     float64       
 10  WALLS CORN B/WHITE 82GR    53 non-null     float64       
 11  WALLS POPULAIRE CKL        53 non-null     float64       
 12  WALLS POPU

In [14]:
# Ubah kolom TANGGAL menjadi datetime
df_final["TANGGAL"] = pd.to_datetime(df_final["TANGGAL"])

# Ubah dari wide ke long format agar cocok untuk plotly express
df_long = df_final.melt(id_vars="TANGGAL", var_name="NAMA BARANG", value_name="QTY")

# Buat line chart
fig = px.line(
    df_long,
    x="TANGGAL",
    y="QTY",
    color="NAMA BARANG",
    title="Jumlah Penjualan per Minggu per Item",
    markers=True,
    labels={"TANGGAL": "Tanggal", "QTY": "Jumlah Terjual", "NAMA BARANG": "Nama Produk"},
    hover_data={"TANGGAL": True, "QTY": True, "NAMA BARANG": True}
)

# Layout tambahan
fig.update_layout(
    legend_title_text='Klik nama produk untuk tampilkan/sembunyikan',
    xaxis_title='Tanggal',
    yaxis_title='Jumlah Terjual',
    template='plotly_white',
    height=500,
    width=900
)

fig.show()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_8072\2313548066.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   TANGGAL                    53 non-null     datetime64[ns]
 1   INDOMIE GRG SPECIAL        53 non-null     float64       
 2   INDOMILK KID CKL 115ML     53 non-null     float64       
 3   SEDAAP MIE SOTO            53 non-null     float64       
 4   SAKINAH 1500ML             53 non-null     float64       
 5   INDOMILK KID VNL 115ML     53 non-null     float64       
 6   INDOMILK KID STR 115ML     53 non-null     float64       
 7   INDOMIE SOTO MIE           53 non-null     float64       
 8   SEDAAP MIE AYAM BWG        53 non-null     float64       
 9   EKONOMI LIQUID REFF 235ML  53 non-null     float64       
 10  WALLS CORN B/WHITE 82GR    53 non-null     float64       
 11  WALLS POPULAIRE CKL        53 non-null     float64       
 12  WALLS POPU

In [16]:
# Mengubah kolom TANGGAL menjadi datetime dan menjadikannya index
df_final['TANGGAL'] = pd.to_datetime(df_final['TANGGAL'])
df_final= df_final.set_index('TANGGAL')
df_final = df_final.asfreq('W-MON') # Menetapkan frekuensi mingguan (Senin)

df_final

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8072\734437237.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,INDOMIE GRG SPECIAL,INDOMILK KID CKL 115ML,SEDAAP MIE SOTO,SAKINAH 1500ML,INDOMILK KID VNL 115ML,INDOMILK KID STR 115ML,INDOMIE SOTO MIE,SEDAAP MIE AYAM BWG,EKONOMI LIQUID REFF 235ML,WALLS CORN B/WHITE 82GR,WALLS POPULAIRE CKL,WALLS POPULAIRE STR,MAMA LEMON 115ML,DELFI TOP CHO 9GR,ROMA KELAPA 300GR,DELFI TOP B/WHITE 9GR,BISKUAT BOLU CKL 16GR,DELFI TOP STR 9GR
TANGGAL,,,,,,,,,,,,,,,,,,
2024-01-01,129.0,3.0,38.0,6.0,8.0,3.0,18.0,14.0,9.0,17.0,10.0,4.0,3.0,8.0,12.0,6.0,5.0,5.0
2024-01-08,119.0,12.0,16.0,6.0,13.0,14.0,13.0,1.0,6.0,12.0,3.0,4.0,6.0,4.0,8.0,7.0,4.0,4.0
2024-01-15,88.0,13.0,20.0,5.0,10.0,10.0,16.0,9.0,4.0,3.0,7.0,7.0,7.0,1.0,6.0,3.0,7.0,3.0
2024-01-22,147.0,18.0,32.0,8.0,14.0,16.0,19.0,3.0,6.0,22.0,7.0,6.0,8.0,2.0,3.0,4.0,5.0,0.0
2024-01-29,114.0,20.0,16.0,4.0,19.0,22.0,15.0,5.0,11.0,8.0,8.0,5.0,7.0,10.0,6.0,9.0,0.0,5.0
2024-02-05,187.0,10.0,24.0,8.0,12.0,10.0,23.0,9.0,8.0,15.0,4.0,6.0,4.0,10.0,6.0,3.0,1.0,2.0
2024-02-12,151.0,18.0,16.0,5.0,25.0,23.0,11.0,5.0,5.0,3.0,9.0,7.0,9.0,5.0,9.0,4.0,0.0,6.0
2024-02-19,123.0,29.0,22.0,7.0,26.0,21.0,14.0,7.0,8.0,9.0,5.0,7.0,3.0,6.0,6.0,1.0,2.0,12.0
2024-02-26,112.0,12.0,20.0,6.0,17.0,11.0,6.0,5.0,10.0,6.0,5.0,2.0,8.0,17.0,12.0,6.0,3.0,9.0


In [17]:
# 1. Mengubah index 'TANGGAL' menjadi kolom biasa
df_final = df_final.reset_index()
df_final.rename(columns={'index': 'TANGGAL'}, inplace=True) # Rename kolom 'index' yang baru dibuat menjadi 'TANGGAL'

# 2. Pastikan kolom 'TANGGAL' adalah tipe datetime (sekarang sudah jadi kolom biasa)
df_final['TANGGAL'] = pd.to_datetime(df_final['TANGGAL'])

# 3. Identifikasi kolom produk
product_columns = [col for col in df_final.columns if col != 'TANGGAL']

In [18]:
# --- Fungsi untuk Menghitung Metrik ---
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Calculate MAPE, handling division by zero for actuals
    # Only include points where y_true is not zero for MAPE calculation
    non_zero_indices = y_true != 0
    if np.sum(non_zero_indices) > 0:
        mape = np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100
    else:
        mape = np.nan # Jika semua nilai aktual adalah 0, MAPE tidak terdefinisi

    return mae, rmse, mape

In [19]:
# --- Persiapan untuk Forecasting ---
results = []
train_size = int(len(df_final) * 0.8)
test_size = len(df_final) - train_size

print("="*80)
print("             MEMULAI FORECASTING DENGAN PROPHET DAN EVALUASI             ")
print("="*80)
print(f"Total data historis: {len(df_final)} titik.")
print(f"Ukuran data training (80%): {train_size} titik.")
print(f"Ukuran data testing (20%): {test_size} titik.")
print("\n")

# Loop melalui setiap kolom produk untuk melakukan forecasting
for product in product_columns:
    print(f"\n--- Memproses Produk: {product} ---")

    # Siapkan DataFrame untuk Prophet: 'ds' (datetime) dan 'y' (nilai numerik)
    df_prophet = df_final[['TANGGAL', product]].copy()
    df_prophet.rename(columns={'TANGGAL': 'ds', product: 'y'}, inplace=True)

    # Membagi data menjadi training dan testing (split time series)
    train_df = df_prophet.iloc[:train_size]
    test_df = df_prophet.iloc[train_size:]

    # Inisialisasi dan latih model Prophet
    # Karena data adalah agregasi mingguan (Tanggal adalah awal minggu),
    # seasonality mingguan internal Prophet tidak relevan.
    # Yearly seasonality juga dimatikan karena hanya ~1 tahun data.
    m = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        growth='linear',
        changepoint_prior_scale=0.05 # Default value
    )
    m.fit(train_df)

    # Buat DataFrame untuk periode waktu yang akan diprediksi (periode test)
    # Ini harus mencakup tanggal-tanggal yang ada di test_df
    future = m.make_future_dataframe(periods=len(test_df), freq='W', include_history=False) # Hanya future dates

    # Lakukan prediksi
    forecast = m.predict(future)

    # Ambil nilai prediksi untuk periode test
    y_true = test_df['y'].values
    y_pred = forecast['yhat'].values

    # Hitung metrik evaluasi
    mae, rmse, mape = calculate_metrics(y_true, y_pred)

    print(f"  - MAE: {mae:.2f}")
    print(f"  - RMSE: {rmse:.2f}")
    if pd.isna(mape):
        print(f"  - MAPE: N/A (Semua nilai aktual adalah 0 di periode testing)")
    else:
        print(f"  - MAPE: {mape:.2f}%")

    # Simpan hasil
    results.append({
        'Product': product,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    })

# Konversi hasil ke DataFrame untuk analisis lebih lanjut
df_results = pd.DataFrame(results)

print("\n")
print("="*80)
print("             RINGKASAN HASIL EVALUASI MODEL FORECASTING             ")
print("="*80)

# Urutkan berdasarkan MAPE (terendah ke tertinggi), lalu MAE
df_results_sorted = df_results.sort_values(by='MAPE', ascending=True)

print(df_results_sorted.to_string(index=False)) # to_string() untuk menampilkan semua baris tanpa truncasi

print("\n📈 Rata-rata Performa Semua Produk:")
print(f"Avg MAE: {df_results_sorted['MAE'].mean():.2f}")
print(f"Avg RMSE: {df_results_sorted['RMSE'].mean():.2f}")
print(f"Avg MAPE: {df_results_sorted['MAPE'].mean():.2f}%")

print("--- Catatan Penting ---")
print("1. Akurasi model mungkin terbatas karena jumlah data historis yang relatif sedikit (53 titik).")
print("2. 'MAPE: N/A' menunjukkan bahwa semua nilai aktual di periode testing adalah 0.")
print("3. Hasil forecast ini berdasarkan data dummy. Untuk hasil nyata, gunakan data aktual Anda.")

             MEMULAI FORECASTING DENGAN PROPHET DAN EVALUASI             
Total data historis: 53 titik.
Ukuran data training (80%): 42 titik.
Ukuran data testing (20%): 11 titik.



--- Memproses Produk: INDOMIE GRG SPECIAL ---


07:40:48 - cmdstanpy - INFO - Chain [1] start processing
07:40:49 - cmdstanpy - INFO - Chain [1] done processing
07:40:50 - cmdstanpy - INFO - Chain [1] start processing


  - MAE: 29.34
  - RMSE: 34.63
  - MAPE: 33.32%

--- Memproses Produk: INDOMILK KID CKL 115ML ---


07:40:50 - cmdstanpy - INFO - Chain [1] done processing
07:40:50 - cmdstanpy - INFO - Chain [1] start processing


  - MAE: 10.03
  - RMSE: 12.46
  - MAPE: 27.29%

--- Memproses Produk: SEDAAP MIE SOTO ---


...

  - MAE: 3.13
  - RMSE: 3.46
  - MAPE: 128.97%

--- Memproses Produk: BISKUAT BOLU CKL 16GR ---


07:40:58 - cmdstanpy - INFO - Chain [1] done processing
07:40:59 - cmdstanpy - INFO - Chain [1] start processing


  - MAE: 2.41
  - RMSE: 3.41
  - MAPE: 30.04%

--- Memproses Produk: DELFI TOP STR 9GR ---


07:40:59 - cmdstanpy - INFO - Chain [1] done processing


  - MAE: 2.13
  - RMSE: 2.40
  - MAPE: 130.33%


             RINGKASAN HASIL EVALUASI MODEL FORECASTING             
                  Product       MAE      RMSE       MAPE
   INDOMILK KID CKL 115ML 10.026723 12.460547  27.288926
      WALLS POPULAIRE CKL  1.728428  2.068480  27.927054
    BISKUAT BOLU CKL 16GR  2.409274  3.412138  30.038095
      INDOMIE GRG SPECIAL 29.342318 34.628673  33.315869
         INDOMIE SOTO MIE  9.720926 12.447971  42.376840
          SEDAAP MIE SOTO  6.782322  7.760204  45.169871
        DELFI TOP CHO 9GR  2.083466  2.374085  52.213486
           SAKINAH 1500ML 13.406515 15.750651  59.363765
         MAMA LEMON 115ML  1.720983  2.452152  59.566934
        ROMA KELAPA 300GR  3.962875  4.748433  61.139311
      SEDAAP MIE AYAM BWG  2.217071  2.742488  67.668706
  WALLS CORN B/WHITE 82GR  4.568275  6.111001  71.990882
   INDOMILK KID STR 115ML  7.736716  8.695808 104.868908
      WALLS POPULAIRE STR  3.187493  3.504508 112.299710
EKONOMI LIQUID REFF 235ML  